In [1]:
import numpy as np
import os, sys

In [2]:
sys.path.append("../data/dataset")
sys.path.append("..//microstate_lib/code")
from dataset import *

In [3]:
dataset_facade = EEGDatasetFacade(dataset_base_path="../data")

In [4]:
dataset = dataset_facade("epileptic_eeg_dataset")

In [5]:
data = dataset.get_mne_data([10, 1])

Extracting EDF parameters from c:\Users\Micro\Desktop\MainResearchProjects\snn-seizure-prediction-research\data\sEEG\epileptic_eeg_dataset\Raw_EDF_Files\p10_Record1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [6]:
import eeg_recording

In [7]:
recording = eeg_recording.SingleSubjectRecording("0", data)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [8]:
recording.run_latent_kmeans(n_states = 4, use_gfp = True, n_inits = 100)

In [ ]:
recording.gev_tot

np.float64(0.8760361571308962)

In [ ]:
data.load_data()
data.set_eeg_reference('average')

Reading 0 ... 5401499  =      0.000 ... 10802.998 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Measurement date,"July 09, 2015 05:36:39 GMT"
Experimenter,Unknown
Participant,1785-15
Digitized points,Not available
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,1.00 Hz
Lowpass,70.00 Hz


In [ ]:
import asrpy
asr = asrpy.ASR(sfreq=data.info["sfreq"], cutoff=30)

In [ ]:
asr.fit(data)

In [ ]:
data = asr.transform(data)

In [ ]:
data.set_eeg_reference('average')

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Measurement date,"July 09, 2015 05:36:39 GMT"
Experimenter,Unknown
Participant,1785-15
Digitized points,Not available
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,1.00 Hz
Lowpass,70.00 Hz


In [29]:
recording.run_latent_kmeans(n_states = 4, use_gfp = True, n_inits = 100)

In [33]:
recording.gev_tot

np.float64(0.8946838971900718)

In [1]:
import pyprep
import numpy as np
import mne
import os, sys
from datetime import datetime


sys.path.append("../data/dataset")
sys.path.append("..//microstate_lib/code")
from dataset import *
import eeg_recording
import asrpy

In [2]:



dataset_base_path = "../data"
dataset_facade = EEGDatasetFacade(dataset_base_path=dataset_base_path)
dataset = dataset_facade("epileptic_eeg_dataset")


In [3]:

record_indexes = {
    # '10': [[10, 1], [10, 2]], 
    '11': [[11, 1], [11, 2], [11, 3], [11, 4]], 
    '12': [[12, 1], [12, 2], [12, 3]],
    '13': [[13, 1], [13, 2], [13, 3], [13, 4]], 
    '14': [[14, 1], [14, 2], [14, 3]],
    '15': [[15, 1], [15, 2], [15, 3], [15, 4]]
}

In [4]:



microstate_search_range = (4, 16)
n_iters = 100
stop_delta_threshold = 0.025
store_4_microstates = True
save_preprocessed_data = True

store_base_path = dataset.base_path
cut_off = 30

global start_time
global end_time
start_time = -1
end_time = -1
save_segmentation = True

def begin_timing():
    global start_time
    start_time = datetime.now()
def end_timing():
    global end_time
    end_time = datetime.now()
    
def report_execution_time(event = ""):
    end_timing()
    print('[%s] Time Consumption: {}'.format(event, end_time - start_time))


def store(maps, segmentation, gev, preprocessing_desc, person_id):
    n_states = maps.shape[0]
    save_map_file_name = f"[{preprocessing_desc}]person_{person_id}_states{n_states}_gev_{gev}.npy"

    np.save(os.path.join(store_base_path, save_map_file_name), maps)
    if save_segmentation:
        save_segmentation_file_name = f"[seg-{preprocessing_desc}]person_{person_id}_states{n_states}_gev_{gev}.npy"
        np.save(os.path.join(store_base_path, save_segmentation_file_name), segmentation)

load_preprocessing = set(['10'])
    

In [8]:
from pyprep.prep_pipeline import PrepPipeline

prep_params = {
    "ref_chs": "eeg",
    "reref_chs": "eeg",
    "line_freqs": [],
}

montage_kind = "standard_1020"
montage = mne.channels.make_standard_montage(montage_kind)

for person_index in record_indexes:
    print(f"Train microstates for person {person_index}")
    record_index_list = record_indexes[person_index]
    
    pre_gev_tot = 0
    if person_index not in load_preprocessing:
        data_count = len(record_index_list)
        ast_results = []
        for slice_begin in range(0, data_count, 2):
            data = dataset.get_merge_mne_data(record_index_list[slice_begin: slice_begin + 2])
            print({ch_name: ch_name.replace("EEG ", "").replace("-Ref", "") for ch_name in data.ch_names})

            data.rename_channels({ch_name: ch_name.replace("EEG ", "").replace("-Ref", "") for ch_name in data.ch_names})
            data.drop_channels(['ECG EKG', 'Manual'], on_missing='warning')
            #! --- preprocessing ---
            print(f"[Preprocessing 0: PREP]")
            prep = PrepPipeline(data, prep_params, montage)
            prep.fit()

            print(f"[Preprocessing 1: {slice_begin // 2 + 1}/{int(np.ceil(data_count / 2))}]... ASR, cutoff = {cut_off}")
            asr = asrpy.ASR(sfreq=data.info["sfreq"], cutoff=cut_off)
            data.load_data()
            asr.fit(data)
            data = asr.transform(data)
            ast_results.append(data)
        data = mne.concatenate_raws(ast_results)
        del ast_results
        print("[Preprocessing 2]... reference to average")
        data = data.set_eeg_reference('average')
        if save_preprocessed_data:
            mne.export.export_raw(os.path.join(store_base_path, f'[preprocessed]person{person_index}.edf'), data)
    else:
        print(f"Load preprocessed data...")
        data = mne.io.read_raw(os.path.join(store_base_path, f'[preprocessed]person{person_index}.edf'))
    break
    recording = eeg_recording.SingleSubjectRecording("0", data)

    #! --- microstate training
    print(f"Begin training microstates. Result will save in '{store_base_path}'")
    print(f" -- Search Microstate Amount from {microstate_search_range[0]} to {microstate_search_range[1]}")

    for n_states in range(microstate_search_range[0], microstate_search_range[1]):
        print(f"Begin Training {n_states} microstates")
        recording.run_latent_kmeans(n_states = n_states, use_gfp = True, n_inits = n_iters)
        current_gev_tot = recording.gev_tot
        delta = current_gev_tot - pre_gev_tot
        if delta < stop_delta_threshold:
            break
        if n_states == 4 and store_4_microstates:
            store(recording.latent_maps, recording.latent_segmentation, recording.gev_tot, "[avg+asr]", person_index)
        pre_gev_tot = current_gev_tot
    store(recording.latent_maps, recording.latent_segmentation, recording.gev_tot, "[avg+asr]", person_index)
    break

Train microstates for person 11
Extracting EDF parameters from /home/huang/research/snn-seizure-prediction-research/data/sEEG/epileptic_eeg_dataset/Raw_EDF_Files/p11_Record1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/huang/research/snn-seizure-prediction-research/data/sEEG/epileptic_eeg_dataset/Raw_EDF_Files/p11_Record2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


../data/./sEEG/epileptic_eeg_dataset/dataset.py:14: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  return mne.io.read_raw(filepath)
../data/./sEEG/epileptic_eeg_dataset/dataset.py:14: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  return mne.io.read_raw(filepath)
../data/./sEEG/epileptic_eeg_dataset/dataset.py:14: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  return mne.io.read_raw(filepath)
../data/./sEEG/epileptic_eeg_dataset/dataset.py:14: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  return mne.io.read_raw(filepath)
../data/./sEEG/epileptic_eeg_dataset/dataset.py:14: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  return mne.io.read_raw(filepath)


{'EEG Fp2-Ref': 'Fp2', 'EEG Fp1-Ref': 'Fp1', 'EEG F8-Ref': 'F8', 'EEG F4-Ref': 'F4', 'EEG Fz-Ref': 'Fz', 'EEG F3-Ref': 'F3', 'EEG F7-Ref': 'F7', 'EEG A2-Ref': 'A2', 'EEG T4-Ref': 'T4', 'EEG C4-Ref': 'C4', 'EEG Cz-Ref': 'Cz', 'EEG C3-Ref': 'C3', 'EEG T3-Ref': 'T3', 'EEG A1-Ref': 'A1', 'EEG T6-Ref': 'T6', 'EEG P4-Ref': 'P4', 'EEG Pz-Ref': 'Pz', 'EEG P3-Ref': 'P3', 'EEG T5-Ref': 'T5', 'EEG O2-Ref': 'O2', 'EEG O1-Ref': 'O1', 'ECG EKG': 'ECG EKG', 'Manual': 'Manual'}
[Preprocessing 0: PREP]
Reading 0 ... 8893999  =      0.000 ... 17787.998 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
-

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.0s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.0s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
